In [ ]:
# find . -name "*.jpg" -type f
# while read filename; do mv ${filename}.jpg val/${filename}.jpg; done < val.txt


In [121]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import xgboost as xgb
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(3),
        transforms.RandomRotation(3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'btc-trading-patterns'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
#                                           shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=4,
#                                          shuffle=False, num_workers=2)
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model= models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.1)
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=4)

Epoch 0/3
----------


In [117]:

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
features = model._modules.get('avgpool')
model.eval()
def get_vector(input):
    
    my_embedding = torch.zeros(1,512,1,1)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = features.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(input)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding.reshape(512)

In [118]:
#make train and test dmatrix
train_data=[]
train_labels=[]
test_data=[]
test_labels=[]
for inputs,labels in dataloaders['train']:
    feat=get_vector(inputs).numpy().tolist()
    train_data.append(feat)
    train_labels.append(labels)
for inputs,labels in dataloaders['val']:
    feat=get_vector(inputs).numpy().tolist()
    test_data.append(feat)
    test_labels.append(labels)
train_data=np.array(train_data)
train_labels=np.array(train_labels)
test_data=np.array(test_data)
test_labels=np.array(test_labels)
dtrain=xgb.DMatrix(train_data, label=train_labels)
dtest=xgb.DMatrix(test_data, label=test_labels)

    

In [119]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'error'
evallist = [(dtest, 'eval'), (dtrain, 'train')]


In [120]:
num_round = 20
bst = xgb.train(param, dtrain, num_round, evallist)
#bst=bst.best_iteration


[0]	eval-error:0.428571	train-error:0.356394
[1]	eval-error:0.428571	train-error:0.354298
[2]	eval-error:0.412698	train-error:0.314465
[3]	eval-error:0.428571	train-error:0.280922
[4]	eval-error:0.380952	train-error:0.257862
[5]	eval-error:0.444444	train-error:0.228512
[6]	eval-error:0.47619	train-error:0.215933
[7]	eval-error:0.412698	train-error:0.199161
[8]	eval-error:0.428571	train-error:0.188679
[9]	eval-error:0.396825	train-error:0.178197
[10]	eval-error:0.380952	train-error:0.161426
[11]	eval-error:0.412698	train-error:0.163522
[12]	eval-error:0.396825	train-error:0.146751
[13]	eval-error:0.412698	train-error:0.132075
[14]	eval-error:0.412698	train-error:0.109015
[15]	eval-error:0.412698	train-error:0.111111
[16]	eval-error:0.412698	train-error:0.102725
[17]	eval-error:0.396825	train-error:0.096436
[18]	eval-error:0.428571	train-error:0.069182
[19]	eval-error:0.412698	train-error:0.054507


In [95]:
#bst.save_model('75%ontwinsusingresnet.model')

In [105]:
# bst = xgb.Booster({'nthread': 4})  # init model
# bst.load_model('75%ontwinsusingresnet.model')  # load data

In [113]:
#test_acc
testpreds=bst.predict(dtest)

testpreds=np.array([round(i) for i in testpreds])
print(testpreds)
correct=0.0
for i in range(testpreds.shape[0]):
    if testpreds[i]==test_labels[i]:
        correct+=1.0
print(correct/testpreds.shape[0])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.]
0.6825396825396826


In [61]:
test_labels

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])

In [64]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

In [65]:
model= models.alexnet(pretrained=True)
num_ftrs = model.classifier[6].in_features
features = list(model.classifier.children())[:-1]
features.extend([nn.Linear(num_ftrs, 2)])
model.classifier = nn.Sequential(*features)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.1)
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

Epoch 0/14
----------
train Loss: nan Acc: 0.5618
val Loss: nan Acc: 0.6667

Epoch 1/14
----------


Process Process-297:
Process Process-300:
Process Process-298:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessin

RuntimeError: DataLoader worker (pid 67196) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
features = model.classifier[-2]
model.eval()
def get_vector2(input):
    
    my_embedding = torch.zeros(1,4096)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
        #print(o)
    # 5. Attach that function to our selected layer
    h = features.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(input)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding.reshape(4096)


In [ ]:
train_data=[]
train_labels=[]
test_data=[]
test_labels=[]
for inputs,labels in dataloaders['train']:
    feat=get_vector2(inputs).numpy().tolist()
    train_data.append(feat)
    train_labels.append(labels)
for inputs,labels in dataloaders['val']:
    feat=get_vector2(inputs).numpy().tolist()
    test_data.append(feat)
    test_labels.append(labels)
train_data=np.array(train_data)
train_labels=np.array(train_labels)
test_data=np.array(test_data)
test_labels=np.array(test_labels)
dtrain=xgb.DMatrix(train_data, label=train_labels)
dtest=xgb.DMatrix(test_data, label=test_labels)

In [ ]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'error'
evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [ ]:
num_round = 20
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
testpreds=bst.predict(dtest)

testpreds=np.array([round(i) for i in testpreds])
print(testpreds)
correct=0.0
for i in range(testpreds.shape[0]):
    if testpreds[i]==test_labels[i]:
        correct+=1.0
print(correct/testpreds.shape[0])